# Imports

In [50]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import datetime as dt
import os
import matplotlib.pyplot as plt
import phonecalls as pc
import scipy.stats as stats
import scipy.interpolate as interpol
import statsmodels.api as sm
import pymannkendall as mk
import time
import copy
import math
import pickle


markers = ['o', 'v', 's', 'P', '*', 'X', 'd', 'x', 'D', '2', '+']
colors = ['#348ABD', '#A60628', '#7A68A6', '#467821', '#D55E00', '#CC79A7', '#56B4E9', '#009E73', '#F0E442', '#0072B2']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Unfiltered Data
In practice, this means that the data is just as it was provided, in terms of the phone calls and their temporal limits. No filter is placed regarding the number of calls or the point in time (calendar days). There may be some modifications to anonymize the data.

## UK: Calls

In [2]:
# tmp1 = pc.allcalls("../ukdata/collected_dunbar_calls_v3_e26OK_sorted.csv", (3, 'call'), 1, 2, [6,7], '%d/%m/%y %H:%M', header=False, min_activity=1)
# print(tmp1.dtypes)
# tmp1.head()

In [13]:
tmp2 = pd.read_excel("../ukdata/participant_info.xls", header=1, dtype={'Mobile Number': str})
tmp2.head()

,Subject Number,Initials,Gender,Mobile Number,University/Plans,Stayed in Sheffield?,Subject,Completed Study to T3?
0,1,AH,Male,07805206865,University of Manchester,No,Sociology,Yes
1,2,EF,Male,07905656488,Gap - Sheffield until July 07,Yes,Not applicable,Yes
2,3,LJ,Male,07800759083,University of York,No,Biology,Yes
3,4,DA,Male,07779270485,Leeds Metropolian University,No,Sports Buisness Management,Yes
4,5,MJ,Male,07855941005,Newcastle University,No,Geography,Yes


In [51]:
egoinfo = {}
for i in tmp2.index:
    pnum = tmp2.at[i, 'Mobile Number'][-10:]
    eid = tmp2.at[i, 'Subject Number']
    if tmp2.at[i, 'Gender'] == 'Male':
        efemale = 0
    else:
        efemale = 1
    egoinfo[pnum] = (eid, efemale)

In [ ]:
len(egoinfo.keys())

In [ ]:
egoinfo

In [ ]:
len(str(tmp2.at[0, 'Mobile Number']))

In [ ]:
len(tmp1.at[306132, 'ego'][-11:])

In [ ]:
tmp1.at[306132, 'ego'][-11:]

In [ ]:
tmp2['Mobile Number'] = tmp2['Mobile Number'].map(lambda i: str(i))
eidd = {tmp2.at[i, 'Mobile Number']: tmp2.at[i, 'Subject Number'] for i in tmp2.index}

In [ ]:
# tmp1['egoid'] = tmp1['ego'].map(lambda i: eidd[i[-10:]])
# tmp.to_csv("uktmp.csv", index=False, header=False, sep=",")
# tmp1.head()

In [ ]:
tmp2['efemale'] = tmp2['Gender'].map(lambda i: 0 if i == 'Male' else 1)
tmp2.head()

In [ ]:
esex = {tmp2.at[i, 'Subject Number']: tmp2.at[i, 'efemale'] for i in tmp2.index}

In [ ]:
# tmp1['efemale'] = tmp1['egoid'].map(lambda i: esex[i])
# tmp1.head()

In [ ]:
tmp1['a8'] = tmp1['alter'].map(lambda i: i[-8:])
tmp1.head()

In [ ]:
# cols = ['aid', 'afemale', 'related', 'sameGen', 'ec1', 'ec2', 'ec3', 'alter1', 'alter2', 'alter3']
# t0 = dt.datetime.now()
# uk = pd.DataFrame()
# for eid in range(1, 31):
#     thisego = pd.DataFrame()
#     df1 = tmp1.loc[tmp1['egoid'] == eid]
#     df2 = pd.read_csv(f"../ukdata/data_to_Eduardo/egos_attributeSummaries/ego{eid}_attributeSummary.dat", na_values=-999, header=None, names=cols, sep=" ", dtype={'alter1': str, 'alter2': str, 'alter3': str})
#     for aid in df2.index:
#         nums = [df2.at[aid, col] for col in ['alter1', 'alter2', 'alter3'] if pd.isnull(df2.at[aid, col]) == False]
#         if len(thisego) > 0:
#             nums = [n for n in nums if n not in thisego['a8'].unique()]
#         df3 = df1[df1['a8'].isin(nums)].copy()
#         df3['aid'] = aid
#         df4 = df3.merge(df2[['aid', 'afemale', 'related', 'sameGen', 'ec1', 'ec2', 'ec3']], on='aid', how='left')
#         thisego = thisego.append(df4)
#     uk = uk.append(thisego)
# print(f'Took {dt.datetime.now() - t0}')

In [ ]:
uk['efemale'].value_counts()

In [ ]:
len(uk['pair'].unique())

In [ ]:
tmp1['efemale'].value_counts()

In [ ]:
len(tmp1['pair'].unique())

In [ ]:
uk.loc[(uk['egoid'] == 1) & (uk['aid'] == 5)]

In [ ]:
tmp1.loc[tmp1['a8'] == '47717624']

In [ ]:
df2 = pd.read_csv(f"../ukdata/data_to_Eduardo/egos_attributeSummaries/ego1_attributeSummary.dat", na_values=-999, header=None, names=cols, sep=" ", dtype={'alter1': str, 'alter2': str, 'alter2': str})
df2.head(6)

In [ ]:
df1 = tmp1.loc[tmp1['egoid'] == 1]

In [ ]:
df1.loc[df1['a8'] == '47717624']

In [ ]:
eidd

In [ ]:
nums = [df2.at[5, col] for col in ['alter1', 'alter2', 'alter3'] if pd.isnull(df2.at[5, col]) == False]
nums

In [ ]:
# tmp1 = tmp1.sort_values(by=['ego', 'alter', 'time']).reset_index(drop=True)
# tmp1.to_csv("data/UKc.txt", index=False, header=True, sep="\t")

In [ ]:
# with open("data/UKc.pkl", 'wb') as f:
#     pickle.dump(tmp1, f)
#     f.close()

### Potential problem
Since I detected that some numbers for alters do not match their egos, I went back to the original files and tried to identify which cases are these.

In [ ]:
with open("../ukdata/bills/AA_07974-937603.txt", 'r') as file:
    for line in file.readlines():
        a = line.rsplit()
        if len(a) == 1:
            print(a)
        elif len(a) == 9:
            print(a)
        else:
            print(a)

## UK: Texts

In [ ]:
tmp1 = pc.allcalls("../ukdata/collected_dunbar_calls_v3_e26OK_sorted.csv", (3, 'text'), 1, 2, [6,7], '%d/%m/%y %H:%M', header=False, min_activity=1)
tmp1.head()

In [ ]:
max(tmp1['t'])

In [ ]:
# tmp1 = tmp1.sort_values(by=['ego', 'alter', 'time']).reset_index(drop=True)
# tmp1.to_csv("data/UKt.txt", index=False, header=True, sep="\t")

In [ ]:
# with open("data/UKt.pkl", 'wb') as f:
#     pickle.dump(tmp1, f)
#     f.close()

## US: Calls

In [ ]:
tmp1 = pc.allcalls('../usadata/CallLog.csv', ('type', 'outgoing', 'outgoing+', 'missed'), 'participantID.A', 'number.hash', 
                                ['local_time'], "%Y-%m-%d %H:%M:%S", header=True, min_activity=1)
tmp1.head()

In [ ]:
len(tmp1['pair'].unique())

In [ ]:
# tmp1 = tmp1.sort_values(by=['ego', 'alter', 'time']).reset_index(drop=True)
# tmp1.to_csv("data/USc.txt", index=False, header=True, sep="\t")

In [ ]:
# with open("data/USc.pkl", 'wb') as f:
#     pickle.dump(tmp1, f)
#     f.close()

### US with duration 

In [ ]:
testing = pd.read_csv('../usadata/CallLog.csv')
testing['pair'] = list(zip(testing['participantID.A'], testing['number.hash']))
print(len(testing['pair'].unique()))
testing.head()

In [ ]:
newdf = testing[testing['type'].isin(['outgoing', 'outgoing+'])]
len(newdf['pair'].unique())

In [ ]:
newdf['time'] = pd.to_datetime(newdf['local_time'], format="%Y-%m-%d %H:%M:%S")
newdf['date'] = newdf['time'].map(lambda i: i.strftime('%Y-%m-%d'))
newdf['date'] = pd.to_datetime(newdf['date'], format='%Y-%m-%d')
newdf.head()

In [ ]:
newdf.columns = ['ego', 'dwcfwdcw', 'dwfwefwefwe', 'dwfwefwefwe', 'duration', 'alter', 'pair', 'time', 'date']
newdf = newdf[['ego', 'alter', 'pair', 'duration', 'time', 'date']]
newdf.head()

In [ ]:
newdf = newdf.sort_values(by=['alter', 'ego', 'time'], ascending=[True, True, True]).reset_index(drop=True)
mindate = min(newdf['date'])
newdf['t'] = newdf['date'].map(lambda i: (i - mindate).days)
t0i = newdf.groupby('pair')['date'].min().rename({'date': 't0i'}, axis='columns')
newdf['a'] = newdf.index.map(lambda i: (newdf.at[i, 'date'] - t0i[newdf.at[i, 'pair']]).days)
newdf.head()

In [ ]:
newdf['ego'] = newdf['ego'].map(lambda i: hex(hash(i)))
newdf['alter'] = newdf['alter'].map(lambda i: hex(hash(i)))
newdf['pair'] = newdf['pair'].map(lambda i: hex(hash(i)))
newdf.head()

In [ ]:
hex(hash('fa10-01-19'))

In [ ]:
us = newdf.copy(deep=True)

In [ ]:
us.sort_values(by=['alter', 'ego', 'time'], ascending=[True, True, True], inplace=True)
t0i = us.groupby('ego')['date'].min()
us['tau'] = us.index.map(lambda i: (us.at[i, 'date'] - t0i[us.at[i, 'ego']]).days)
us.head()

In [ ]:
us = us[['ego', 'alter', 'pair', 'duration', 'time', 'date', 't', 'tau', 'a']].reset_index(drop=True)
us.head()

In [ ]:
[min(us['t']), max(us['t']), min(us['a']), max(us['a'])]

In [ ]:
len(us['pair'].unique())

In [ ]:
# us.to_csv("data/USc.txt", index=False, header=True, sep="\t")

### Personality Traits

In [ ]:
pt = pd.read_csv("../usadata/DataRelease/SurveyBig5.csv")
pt.columns

In [ ]:
pt = pt.rename(columns={'Unnamed: 0': 'ego'})
pt.head()

In [ ]:
for col in [2,6,8,9,12,18,21,23,24,27,31,34,35,37,41,43]:
    pt[f'Q{col}'] = 6 - pt[f'Q{col}']
pt.head()

In [ ]:
pt['ego'] = pt['ego'].map(lambda i: i.lower())
pt.head()

In [ ]:
hex(hash(pt.at[0, 'ego'])) in  us['ego'].unique()

In [ ]:
pt['ego'] = pt['ego'].map(lambda i: hex(hash(i)))

In [ ]:
pt['extraversion'] = pt[[f'Q{col}' for col in [1,6,11,16,21,26,31,36]]].sum(axis=1).astype(int)
pt.head()

In [ ]:
pt['agreeableness'] = pt[[f'Q{col}' for col in [2,7,12,17,22,27,32,37,42]]].sum(axis=1).astype(int)
pt['conscientiousness'] = pt[[f'Q{col}' for col in [3,8,13,18,23,28,33,38,43]]].sum(axis=1).astype(int)
pt['neuroticism'] = pt[[f'Q{col}' for col in [4,9,14,19,24,29,34,39]]].sum(axis=1).astype(int)
pt['openness'] = pt[[f'Q{col}' for col in [5,10,15,20,25,30,35,40,41,44]]].sum(axis=1).astype(int)

In [ ]:
pt.head()

In [ ]:
colnames = pt.iloc[:, -5:].columns
personality = {}
personality['us'] = pd.DataFrame({col: list(pt[col]) for col in colnames}, index=pt['ego'])
personality['us'].head()

In [ ]:
for i in personality['us'].index:
    if i not in us['ego'].unique():
        print(f'Problem with {i}')

## US: Texts

In [ ]:
len(tmp1['pair'].unique())

In [ ]:
tmp = pd.read_csv("../usadata/SMSLog.csv")
tmp.tail()

In [ ]:
tmp['type'].value_counts()

In [ ]:
newdf = tmp[tmp['type'] == 'outgoing']
newdf['pair'] = list(zip(newdf['participantID.A'], newdf['number.hash']))
len(newdf['pair'].unique())

In [ ]:
newdf['time'] = pd.to_datetime(newdf['local_time'], format="%Y-%m-%d %H:%M:%S")
newdf['date'] = newdf['time'].map(lambda i: i.strftime('%Y-%m-%d'))
newdf['date'] = pd.to_datetime(newdf['date'], format='%Y-%m-%d')
newdf.head()

In [ ]:
newdf.columns = ['ego', 'dwcfwdcw', 'dwfwefwefwe', 'dwfwefwefwe', 'alter', 'pair', 'time', 'date']
newdf = newdf[['ego', 'alter', 'pair', 'time', 'date']]
newdf.head()

In [ ]:
newdf = newdf.sort_values(by=['alter', 'ego', 'time'], ascending=[True, True, True]).reset_index(drop=True)
mindate = min(newdf['date'])
newdf['t'] = newdf['date'].map(lambda i: (i - mindate).days)
t0i = newdf.groupby('pair')['date'].min().rename({'date': 't0i'}, axis='columns')
newdf['a'] = newdf.index.map(lambda i: (newdf.at[i, 'date'] - t0i[newdf.at[i, 'pair']]).days)
newdf.head()

In [ ]:
newdf['ego'] = newdf['ego'].map(lambda i: hex(hash(i)))
newdf['alter'] = newdf['alter'].map(lambda i: hex(hash(i)))
newdf['pair'] = newdf['pair'].map(lambda i: hex(hash(i)))
newdf.head()

In [ ]:
ust = newdf.copy(deep=True)
[min(ust['t']), max(ust['t']), min(ust['a']), max(ust['a'])]

In [ ]:
ust.sort_values(by=['alter', 'ego', 'time'], ascending=[True, True, True], inplace=True)
t0i = ust.groupby('ego')['date'].min()
ust['tau'] = ust.index.map(lambda i: (ust.at[i, 'date'] - t0i[ust.at[i, 'ego']]).days)

In [ ]:
ust = ust[['ego', 'alter', 'pair', 'time', 'date', 't', 'tau', 'a']].reset_index(drop=True)
ust.head()

In [ ]:
len(ust['pair'].unique())

In [ ]:
# ust.to_csv("data/USt.txt", index=False, header=True, sep="\t")

## Italy

In [ ]:
with open('../itdata/mtl_unfiltered.pickle', 'rb') as handle:
    tmpunf = pickle.load(handle)
    handle.close()
newdf = tmpunf['it'].copy(deep=True)
newdf.head()

In [ ]:
min_activity = 1
newdf['date'] = newdf['time'].map(lambda i: i.strftime('%Y-%m-%d'))
newdf['date'] = pd.to_datetime(newdf['date'], format='%Y-%m-%d')
newdf.sort_values(by=['alter', 'ego', 'time'], ascending=[True, True, True], inplace=True)
mindate = min(newdf['date'])
newdf['t'] = newdf['date'].map(lambda i: (i - mindate).days)
newdf['ea'] = list(zip(newdf['ego'], newdf['alter']))
t0ix = newdf.groupby('ea')['date'].min()
t0i = newdf.groupby('ego')['date'].min()
newdf['a'] = newdf.index.map(lambda i: (newdf.at[i, 'date'] - t0ix[newdf.at[i, 'ea']]).days)
newdf['tau'] = newdf.index.map(lambda i: (newdf.at[i, 'date'] - t0i[newdf.at[i, 'ego']]).days)
ncalls = newdf.groupby('ea')[['time']].count()
ncalls = ncalls.loc[ncalls['time'] >= min_activity]
newdf = newdf[newdf['ea'].isin(ncalls.index)]
newdf['ego'] = newdf['ego'].map(lambda i: hex(hash(i)))
newdf['alter'] = newdf['alter'].map(lambda i: hex(hash(i)))
newdf['ea'] = newdf['ea'].map(lambda i: hex(hash(i)))
newdf.head()

In [ ]:
newdf.columns = ['ego', 'alter', 'time', 'uclock', 'aclock', 'date', 't', 'pair', 'a', 'tau']
newdf = newdf[['ego', 'alter', 'pair', 'time', 'date', 't', 'tau', 'a']]
newdf.head()

In [ ]:
len(newdf['pair'].unique())

In [ ]:
max(newdf['t'])

In [ ]:
# newdf = newdf.sort_values(by=['ego', 'alter', 'time']).reset_index(drop=True)
# newdf.to_csv("data/IT.txt", index=False, header=True, sep="\t")

In [ ]:
# with open("data/IT.pkl", 'wb') as f:
#     pickle.dump(newdf, f)
#     f.close()

### Italian data with extra columns

In [ ]:
with open("../itdata/mtl_withextrainfo.pickle", 'rb') as f:
    it = pickle.load(f)
    f.close()

In [ ]:
it['relationship'].value_counts()

In [ ]:
it.head()

In [ ]:
len(it['pair'].unique())

In [ ]:
newdf = it.copy(deep=True)

In [ ]:
newdf.dtypes

In [ ]:
newdf.sort_values(by=['alter', 'ego', 'time'], ascending=[True, True, True], inplace=True)
t0i = newdf.groupby('ego')['date'].min()
newdf['tau'] = newdf.index.map(lambda i: (newdf.at[i, 'date'] - t0i[newdf.at[i, 'ego']]).days)
newdf['pair'] = newdf['pair'].map(lambda i: hex(hash(i)))
newdf.head()

In [ ]:
# newdf.to_csv("data/IT.txt", index=False, header=True, sep="\t")

### Italy with personality traits

In [ ]:
with open("../itdata/mtl_withsurveyinfo.pickle", 'rb') as f:
    itpt = pickle.load(f)
    f.close()

In [ ]:
[len(newdf['ego'].unique()), len(itpt['ego'].unique())]

In [ ]:
[len(newdf), len(itpt)]

In [ ]:
df = itpt.iloc[:, :8]
df.sort_values(by=['alter', 'ego', 'time'], ascending=[True, True, True], inplace=True)
t0i = df.groupby('ego')['date'].min()
df['tau'] = df.index.map(lambda i: (df.at[i, 'date'] - t0i[df.at[i, 'ego']]).days)
df['pair'] = df['pair'].map(lambda i: hex(hash(i)))
print(len(df))
df.head()

In [ ]:
# df.to_csv("data/IT.txt", index=False, header=True, sep="\t")

In [ ]:
df.iloc[0]['ego']

In [ ]:
result = pd.DataFrame()
for ego in itpt['ego'].unique():
    df0 = itpt.loc[itpt['ego'] == ego]
    df1 = df0.iloc[:, 8:]
    for v in df1.columns:
        result.at[ego, v] = df1.iloc[0][v]
result.head(10)

In [ ]:
personality['it'] = result.copy()

# Import UK data from phone bills

In [52]:
import os
os.listdir("../ukdata/bills/")

['HG_07896-556837.txt',
 'AA_07974-937603.txt',
 'KG_07800-650314.txt',
 'CF_07817-280467.txt',
 'RH_07896-537661.txt',
 'LJ_07800-759083_bills.txt',
 'EF_07905-656488_bills.txt',
 'MI_07855-941005.txt',
 'SS_07814-508939.txt',
 'RF_07791-428359.txt',
 'EHB_07974-937643.txt',
 'LD_07890-948782.txt',
 'DA_07779-270485_bills.txt',
 'CB_07800-756002.txt',
 'FMC_07792-057901.txt',
 'MR_07800-643466.txt',
 'JG_07890-818114.txt',
 'HL_07792-058001.txt',
 'RW_07800-650511.txt',
 'AP_07800-759255.txt',
 'HM_07800-643415.txt',
 'II_07800-756371.txt',
 'MJ_07800-755987.txt',
 'TS_07792-658311.txt',
 'MM_07800-643277.txt',
 'KN_07792-658339.txt',
 'FA_07970-136770.txt',
 'NW_07792-895869.txt',
 'HA_07890-846515.txt',
 'AH_07805-206865_bills.txt']

In [3]:
t0 = dt.datetime.now()
uk = pd.DataFrame()
removed = 0
for bill in os.listdir("../ukdata/bills/"):
    lnum = 0
    with open(f'../ukdata/bills/{bill}', 'r') as file:
        times = []
        dates = []
        alters = []
        durs = []
        iscall = []
        counter = 0
        for i in file.readlines():
            line = i.rsplit()
            if len(line) == 1:
                ego = line[0][-10:]
            elif (len(line) > 3) and (len(line[0]) == 8) and (len(line[1]) == 3) and (len(line[3]) >= 11) and (line[-4] in ['talk', 'text']):
                alter = line[3][-10:]
                if (alter[0] not in [3,8]) and (line[2][2] == ':'):
                    alters.append(alter)
                    datestr = line[0]
                    hourstr = line[2]          
                    dates.append(pd.to_datetime(datestr, format='%d/%m/%y'))
                    times.append(pd.to_datetime(datestr + ' ' + hourstr, format='%d/%m/%y %H:%M'))
                    dur = time.strptime(line[-2], '%H:%M:%S')
                    seconds = (dur.tm_hour * 3600) + (dur.tm_min * 60) + (dur.tm_sec)
                    durs.append(seconds)
                    thistype = line[-4]
                    if thistype == 'talk':
                        iscall.append(1)
                    else:
                        iscall.append(0)
                else:
                    removed += 1
            elif (len(line) > 3) and (line[-4] in ['talk', 'text']):
                alter = line[1][-10:]
                if (alter[0] not in [3,8]) and (line[0][2] == ':'):
                    alters.append(alter)
                    hourstr = line[0]       
                    dates.append(pd.to_datetime(datestr, format='%d/%m/%y'))
                    times.append(pd.to_datetime(datestr + ' ' + hourstr, format='%d/%m/%y %H:%M'))
                    dur = time.strptime(line[-2], '%H:%M:%S')
                    seconds = (dur.tm_hour * 3600) + (dur.tm_min * 60) + (dur.tm_sec)
                    durs.append(seconds)
                    thistype = line[-4]
                    if thistype == 'talk':
                        iscall.append(1)
                    else:
                        iscall.append(0)
                else:
                    removed += 1
            lnum += 1
        anego = pd.DataFrame({'ego': ego, 'alter': alters, 'time': times, 'date': dates, 'isCall': iscall, 'duration': durs})
        uk = uk.append(anego).reset_index(drop=True)
print(f'took {dt.datetime.now() - t0}')

/tmp/ipykernel_30522/570843.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  uk = uk.append(anego).reset_index(drop=True)


took 0:01:59.367281


In [4]:
uk.head()

,ego,alter,time,date,isCall,duration
0,7896556837,7794665331,2007-03-05 15:02:00,2007-03-05,1,56
1,7896556837,7800650511,2007-03-05 16:29:00,2007-03-05,0,0
2,7896556837,7821100539,2007-03-05 16:37:00,2007-03-05,1,28
3,7896556837,7785571995,2007-03-05 16:38:00,2007-03-05,1,30
4,7896556837,7800650511,2007-03-05 16:47:00,2007-03-05,1,2


In [5]:
removed

179

In [6]:
len(uk)

440167

In [7]:
len(uk['alter'].unique())

5915

In [8]:
uk.at[0, 'alter']

'7794665331'

In [9]:
uk['pair'] = list(zip(uk['ego'], uk['alter']))
uk.at[0, 'pair']

('7896556837', '7794665331')

In [10]:
uk.at[0, 'pair']

('7896556837', '7794665331')

In [11]:
uk.head()

,ego,alter,time,date,isCall,duration,pair
0,7896556837,7794665331,2007-03-05 15:02:00,2007-03-05,1,56,"(7896556837, 7794665331)"
1,7896556837,7800650511,2007-03-05 16:29:00,2007-03-05,0,0,"(7896556837, 7800650511)"
2,7896556837,7821100539,2007-03-05 16:37:00,2007-03-05,1,28,"(7896556837, 7821100539)"
3,7896556837,7785571995,2007-03-05 16:38:00,2007-03-05,1,30,"(7896556837, 7785571995)"
4,7896556837,7800650511,2007-03-05 16:47:00,2007-03-05,1,2,"(7896556837, 7800650511)"


In [15]:
for ego in uk['ego'].unique():
    if ego not in egoinfo.keys():
        print('not in spreadsheet', ego)
for ego in egoinfo.keys():
    if ego not in uk['ego'].unique():
        print('not in uk', ego)

not in spreadsheet 7890846515
not in uk 7999103487


In [16]:
uk.loc[uk['ego'] == '7999103487']

,ego,alter,time,date,isCall,duration,pair


In [17]:
uk.loc[uk['ego'] == '7890846515']

,ego,alter,time,date,isCall,duration,pair
398387,7890846515,7933201071,2008-08-07 21:19:00,2008-08-07,0,0,"(7890846515, 7933201071)"
398388,7890846515,7799327129,2008-08-07 21:23:00,2008-08-07,0,0,"(7890846515, 7799327129)"
398389,7890846515,7933201071,2008-08-07 21:28:00,2008-08-07,0,0,"(7890846515, 7933201071)"
398390,7890846515,7933201071,2008-08-07 21:38:00,2008-08-07,0,0,"(7890846515, 7933201071)"
398391,7890846515,7933201071,2008-08-07 21:43:00,2008-08-07,0,0,"(7890846515, 7933201071)"
...,...,...,...,...,...,...,...
438502,7890846515,7792658339,2008-08-07 21:14:00,2008-08-07,0,0,"(7890846515, 7792658339)"
438503,7890846515,7792658339,2008-08-07 21:18:00,2008-08-07,0,0,"(7890846515, 7792658339)"
438504,7890846515,7867861984,2008-08-07 21:26:00,2008-08-07,1,13,"(7890846515, 7867861984)"
438505,7890846515,7531492473,2008-08-07 22:01:00,2008-08-07,1,2,"(7890846515, 7531492473)"


In [18]:
ego26 = pd.read_csv("../ukdata/egos_alterNumbers/ego26_alterNumbers.dat", sep=" ", header=None, dtype={0:str})
ego26['len'] = ego26[0].map(lambda i: len(i))
ego26['len'].value_counts()
ego26['alters'] = ego26[0].map(lambda i: i[-10:])
alters26 = set(ego26['alters'])

In [19]:
egou = uk.loc[uk['ego'] == '7890846515']
altersu = set(egou['alter'])

In [20]:
[len(alters26), len(altersu)]

[71, 607]

In [21]:
count = 0
for i in alters26:
    if i not in altersu:
        count += 1
count

19

In [22]:
egoinfo['7890846515'] = (26, 1)

In [23]:
uk['eid'] = uk['ego'].map(lambda i: egoinfo[i][0])
uk['efemale'] = uk['ego'].map(lambda i: egoinfo[i][1])
uk.head()

,ego,alter,time,date,isCall,duration,pair,eid,efemale
0,7896556837,7794665331,2007-03-05 15:02:00,2007-03-05,1,56,"(7896556837, 7794665331)",6,0
1,7896556837,7800650511,2007-03-05 16:29:00,2007-03-05,0,0,"(7896556837, 7800650511)",6,0
2,7896556837,7821100539,2007-03-05 16:37:00,2007-03-05,1,28,"(7896556837, 7821100539)",6,0
3,7896556837,7785571995,2007-03-05 16:38:00,2007-03-05,1,30,"(7896556837, 7785571995)",6,0
4,7896556837,7800650511,2007-03-05 16:47:00,2007-03-05,1,2,"(7896556837, 7800650511)",6,0


In [23]:
uk.dtypes

ego                 object
alter               object
time        datetime64[ns]
date        datetime64[ns]
isCall               int64
duration             int64
pair                object
eid                  int64
efemale              int64
dtype: object

In [24]:
alterid = {}
for file in os.listdir("../ukdata/egos_alterNumbers/"):
    # print(file)
    eid = int(file[3:5])
    alterid[eid] = {}
    df = pd.read_csv(f"../ukdata/egos_alterNumbers/{file}", header=None, sep=" ", dtype={0: str})
    for i in df.index:
        numb = df.at[i, 0]
        if len(numb) > 10:
            numb = numb[-10:]
            alterid[eid][numb] = int(df.at[i, 1])

In [25]:
for i in range(1, 31):
    if i not in alterid.keys():
        print(i)

7
13
18
19
30


In [26]:
cols = ['aid', 'afemale', 'related', 'sameGen', 'ec1', 'ec2', 'ec3', 'alter1', 'alter2', 'alter3']

for ego in ['07', '13', '18', '19', '30']:
    eid = int(ego)
    alterid[eid] = {}
    df = uk.loc[uk['eid'] == eid]
    a10 = [i for i in df['alter']]
    a8 = [i[-8:] for i in df['alter']]
    df2 = pd.read_csv(f"../ukdata/egos_attributeSummaries/ego{ego}_attributeSummary.dat", na_values=-999, header=None, names=cols, sep=" ", dtype={'alter1': str, 'alter2': str, 'alter3': str})
    for i in df2.index:
        num1 = df2.at[i, 'alter1']
        num2 = df2.at[i, 'alter2']
        num3 = df2.at[i, 'alter3']
        if num1 in a8:
            pos = a8.index(num1)
            number = a10[pos]
            alterid[eid][number] = i
        if (not pd.isna(num2)) and (num2 in a8):
            pos = a8.index(num2)
            number = a10[pos]
            alterid[eid][number] = i
        if (not pd.isna(num3)) and (num3 in a8):
            pos = a8.index(num3)
            number = a10[pos]
            alterid[eid][number] = i

In [27]:
alterid.keys()

dict_keys([10, 9, 24, 28, 22, 16, 6, 17, 21, 15, 26, 29, 3, 2, 12, 14, 1, 4, 27, 8, 20, 5, 25, 11, 23, 7, 13, 18, 19, 30])

In [28]:
alterid[7]

{'7886653124': 0,
 '1142960331': 1,
 '7974414160': 1,
 '1142967595': 9,
 '7886935602': 10,
 '1246550323': 24,
 '1142553990': 15,
 '7753598894': 18,
 '1142641985': 18,
 '7792658311': 21,
 '7731512955': 22,
 '7929885391': 23,
 '1142437671': 29,
 '7870886446': 28,
 '1142556721': 30,
 '7902793964': 31,
 '7792658339': 33,
 '7969760845': 34,
 '7835588869': 35,
 '7970136770': 39,
 '7986312152': 40,
 '7767631720': 41,
 '7908803983': 42,
 '7760150929': 43,
 '7709186078': 44,
 '7788653124': 45,
 '7800643466': 46,
 '7706863788': 47,
 '7796015648': 48,
 '7791018584': 49,
 '7772378467': 50,
 '7807099302': 51,
 '1142500254': 52,
 '7854098713': 53,
 '7891656476': 54,
 '7714413317': 55,
 '7877543792': 56,
 '1142585103': 57,
 '1142684753': 58,
 '1142490019': 59,
 '7828717709': 60,
 '7737278047': 61,
 '7973100123': 62,
 '1142931761': 63,
 '7852796054': 64,
 '7977115946': 65,
 '7800714259': 66,
 '7800643415': 67,
 '7800643277': 68,
 '7830044389': 69,
 '1709373715': 70,
 '7946866841': 71,
 '7890948782': 7

In [29]:
face = pd.read_excel("../ukdata/face_to_face contact.xlsx", na_values=999, dtype={'Alter': 'Int64'})
face.dtypes

Ego                  int64
Alter                Int64
Alter mobile        object
Alter landline      object
Face contact T1    float64
Face contact T2    float64
Face contact T3    float64
dtype: object

In [30]:
def dealnum(num):
    if type(num) == str:
        num2 = num.replace(" ", "")[-10:]
    elif type(num) == int:
        num2 = str(num)[-10:]
    else:
        num2 = np.nan
    return num2

In [31]:
face['amob'] = face['Alter mobile'].map(lambda i: dealnum(i))
face['aland'] = face['Alter landline'].map(lambda i: dealnum(i))
face.head()

,Ego,Alter,Alter mobile,Alter landline,Face contact T1,Face contact T2,Face contact T3,amob,aland
0,2,201,NaN,0114 2661804,1.0,1.0,1.0,NaN,1142661804
1,2,202,07986 687851,0114 2661804,1.0,1.0,1.0,7986687851,1142661804
2,2,203,07847 628716,NaN,1.0,1.0,1.0,7847628716,NaN
3,2,204,07962 022353,01246 235985,22.5,23.0,5.0,7962022353,1246235985
4,2,205,07962 022353,01246 235985,18.0,23.0,5.0,7962022353,1246235985


In [32]:
for i in face.index:
    ego = face.at[i, 'Ego']
    amob = face.at[i, 'amob']
    if pd.isna(amob):
        num = face.at[i, 'aland']
        if (not pd.isna(num)) and (num in alterid[ego].keys()):
            face.at[i, 'aid'] = alterid[ego][num]
        else:
            face.at[i, 'aid'] = np.nan
    elif amob in alterid[ego].keys():
        face.at[i, 'aid'] = alterid[ego][amob]
    else:
        face.at[i, 'aid'] = np.nan
face['aid'] = face['aid'].astype('Int64')
face.head()

,Ego,Alter,Alter mobile,Alter landline,Face contact T1,Face contact T2,Face contact T3,amob,aland,aid
0,2,201,NaN,0114 2661804,1.0,1.0,1.0,NaN,1142661804,2
1,2,202,07986 687851,0114 2661804,1.0,1.0,1.0,7986687851,1142661804,1
2,2,203,07847 628716,NaN,1.0,1.0,1.0,7847628716,NaN,2
3,2,204,07962 022353,01246 235985,22.5,23.0,5.0,7962022353,1246235985,13
4,2,205,07962 022353,01246 235985,18.0,23.0,5.0,7962022353,1246235985,13


In [33]:
face.columns = ['eid', 'fdfkwjbnfjkwr', 'eqdfeqd', 'fewfwefwe', 'face1', 'face2', 'face3', 'edffge', 'retrew', 'aid']
face['pair3'] = list(zip(face['eid'], face['aid']))
face = face[['pair3', 'face1', 'face2', 'face3']]
face.head()

,pair3,face1,face2,face3
0,"(2, 2)",1.0,1.0,1.0
1,"(2, 1)",1.0,1.0,1.0
2,"(2, 2)",1.0,1.0,1.0
3,"(2, 13)",22.5,23.0,5.0
4,"(2, 13)",18.0,23.0,5.0


In [34]:
for i in range(1, 31):
    if i not in alterid.keys():
        print(i)

In [35]:
uk['pair2'] = list(zip(uk['eid'], uk['alter']))
uk['aid'] = uk['pair2'].map(lambda i: int(alterid[i[0]][i[1]]) if i[1] in alterid[i[0]].keys() else np.nan).astype('Int64')
uk.head()

,ego,alter,time,date,isCall,duration,pair,eid,efemale,pair2,aid
0,7896556837,7794665331,2007-03-05 15:02:00,2007-03-05,1,56,"(7896556837, 7794665331)",6,0,"(6, 7794665331)",6
1,7896556837,7800650511,2007-03-05 16:29:00,2007-03-05,0,0,"(7896556837, 7800650511)",6,0,"(6, 7800650511)",63
2,7896556837,7821100539,2007-03-05 16:37:00,2007-03-05,1,28,"(7896556837, 7821100539)",6,0,"(6, 7821100539)",0
3,7896556837,7785571995,2007-03-05 16:38:00,2007-03-05,1,30,"(7896556837, 7785571995)",6,0,"(6, 7785571995)",<NA>
4,7896556837,7800650511,2007-03-05 16:47:00,2007-03-05,1,2,"(7896556837, 7800650511)",6,0,"(6, 7800650511)",63


In [36]:
uk['pair3'] = list(zip(uk['eid'], uk['aid']))
uk.head()

,ego,alter,time,date,isCall,duration,pair,eid,efemale,pair2,aid,pair3
0,7896556837,7794665331,2007-03-05 15:02:00,2007-03-05,1,56,"(7896556837, 7794665331)",6,0,"(6, 7794665331)",6,"(6, 6)"
1,7896556837,7800650511,2007-03-05 16:29:00,2007-03-05,0,0,"(7896556837, 7800650511)",6,0,"(6, 7800650511)",63,"(6, 63)"
2,7896556837,7821100539,2007-03-05 16:37:00,2007-03-05,1,28,"(7896556837, 7821100539)",6,0,"(6, 7821100539)",0,"(6, 0)"
3,7896556837,7785571995,2007-03-05 16:38:00,2007-03-05,1,30,"(7896556837, 7785571995)",6,0,"(6, 7785571995)",<NA>,"(6, <NA>)"
4,7896556837,7800650511,2007-03-05 16:47:00,2007-03-05,1,2,"(7896556837, 7800650511)",6,0,"(6, 7800650511)",63,"(6, 63)"


In [37]:
len(uk) - uk['aid'].isna().sum()

316900

In [38]:
uk = uk.dropna(subset=['aid'])
len(uk)

316900

In [39]:
alterextra = {}
for file in os.listdir("../ukdata/egos_attributeSummaries/"):
    if file[-4:] == '.dat':
        eid = int(file[3:5])
        alterextra[eid] = {}
        df = pd.read_csv(f"../ukdata/egos_attributeSummaries/{file}", na_values=-999, header=None, names=cols, sep=" ", dtype={'alter1': str, 'alter2': str, 'alter3': str}, on_bad_lines='skip')
        df = df[['afemale', 'related', 'sameGen', 'ec1', 'ec2', 'ec3']]
        tuples = df.apply(tuple,axis=1).to_numpy()
        for aid in range(len(tuples)):
            alterextra[eid][aid] = tuples[aid]

In [40]:
for i in alterextra.keys():
    print(i, len(alterextra[i].keys()))

29 128
1 71
30 273
10 160
9 203
2 142
4 246
16 452
24 142
13 378
11 228
23 151
28 340
17 441
26 661
7 261
21 244
5 204
6 331
19 274
20 239
25 123
14 615
8 141
27 265
15 789
18 268
22 236
12 303
3 185


In [41]:
uk.head()

,ego,alter,time,date,isCall,duration,pair,eid,efemale,pair2,aid,pair3
0,7896556837,7794665331,2007-03-05 15:02:00,2007-03-05,1,56,"(7896556837, 7794665331)",6,0,"(6, 7794665331)",6,"(6, 6)"
1,7896556837,7800650511,2007-03-05 16:29:00,2007-03-05,0,0,"(7896556837, 7800650511)",6,0,"(6, 7800650511)",63,"(6, 63)"
2,7896556837,7821100539,2007-03-05 16:37:00,2007-03-05,1,28,"(7896556837, 7821100539)",6,0,"(6, 7821100539)",0,"(6, 0)"
4,7896556837,7800650511,2007-03-05 16:47:00,2007-03-05,1,2,"(7896556837, 7800650511)",6,0,"(6, 7800650511)",63,"(6, 63)"
5,7896556837,1142551732,2007-03-05 16:48:00,2007-03-05,1,143,"(7896556837, 1142551732)",6,0,"(6, 1142551732)",7,"(6, 7)"


In [42]:
uk['afemale'] = uk['pair3'].map(lambda i: alterextra[i[0]][i[1]][0])
uk['related'] = uk['pair3'].map(lambda i: alterextra[i[0]][i[1]][1])
uk['sameGen'] = uk['pair3'].map(lambda i: alterextra[i[0]][i[1]][2])
uk['ec1'] = uk['pair3'].map(lambda i: alterextra[i[0]][i[1]][3])
uk['ec2'] = uk['pair3'].map(lambda i: alterextra[i[0]][i[1]][4])
uk['ec3'] = uk['pair3'].map(lambda i: alterextra[i[0]][i[1]][5])
uk.head()

,ego,alter,time,date,isCall,duration,pair,eid,efemale,pair2,aid,pair3,afemale,related,sameGen,ec1,ec2,ec3
0,7896556837,7794665331,2007-03-05 15:02:00,2007-03-05,1,56,"(7896556837, 7794665331)",6,0,"(6, 7794665331)",6,"(6, 6)",1.0,0.0,NaN,9.0,8.0,8.0
1,7896556837,7800650511,2007-03-05 16:29:00,2007-03-05,0,0,"(7896556837, 7800650511)",6,0,"(6, 7800650511)",63,"(6, 63)",1.0,0.0,NaN,3.0,4.0,1.0
2,7896556837,7821100539,2007-03-05 16:37:00,2007-03-05,1,28,"(7896556837, 7821100539)",6,0,"(6, 7821100539)",0,"(6, 0)",0.0,1.0,0.0,9.0,10.0,9.0
4,7896556837,7800650511,2007-03-05 16:47:00,2007-03-05,1,2,"(7896556837, 7800650511)",6,0,"(6, 7800650511)",63,"(6, 63)",1.0,0.0,NaN,3.0,4.0,1.0
5,7896556837,1142551732,2007-03-05 16:48:00,2007-03-05,1,143,"(7896556837, 1142551732)",6,0,"(6, 1142551732)",7,"(6, 7)",1.0,0.0,NaN,9.0,8.0,8.0


In [43]:
uk['related'].value_counts()

0.0    246857
1.0     35314
Name: related, dtype: int64

In [44]:
print(len(uk['pair3'].unique()))
uk = uk.merge(face, on='pair3', how='left')
print(len(uk['pair3'].unique()))
uk.head()

2277
2277


,ego,alter,time,date,isCall,duration,pair,eid,efemale,pair2,...,pair3,afemale,related,sameGen,ec1,ec2,ec3,face1,face2,face3
0,7896556837,7794665331,2007-03-05 15:02:00,2007-03-05,1,56,"(7896556837, 7794665331)",6,0,"(6, 7794665331)",...,"(6, 6)",1.0,0.0,NaN,9.0,8.0,8.0,1.0,1.0,1.0
1,7896556837,7800650511,2007-03-05 16:29:00,2007-03-05,0,0,"(7896556837, 7800650511)",6,0,"(6, 7800650511)",...,"(6, 63)",1.0,0.0,NaN,3.0,4.0,1.0,5.0,124.0,286.0
2,7896556837,7821100539,2007-03-05 16:37:00,2007-03-05,1,28,"(7896556837, 7821100539)",6,0,"(6, 7821100539)",...,"(6, 0)",0.0,1.0,0.0,9.0,10.0,9.0,1.0,1.0,1.0
3,7896556837,7800650511,2007-03-05 16:47:00,2007-03-05,1,2,"(7896556837, 7800650511)",6,0,"(6, 7800650511)",...,"(6, 63)",1.0,0.0,NaN,3.0,4.0,1.0,5.0,124.0,286.0
4,7896556837,1142551732,2007-03-05 16:48:00,2007-03-05,1,143,"(7896556837, 1142551732)",6,0,"(6, 1142551732)",...,"(6, 7)",1.0,0.0,NaN,9.0,8.0,8.0,1.0,1.0,133.0


In [45]:
sum(~uk['face1'].isna())

202366

## Include layers

In [48]:
import pyreadstat

ImportError: dlopen(/Users/vsvh/Library/Python/3.10/lib/python/site-packages/pyreadstat/pyreadstat.cpython-310-darwin.so, 0x0002): tried: '/Users/vsvh/Library/Python/3.10/lib/python/site-packages/pyreadstat/pyreadstat.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/vsvh/Library/Python/3.10/lib/python/site-packages/pyreadstat/pyreadstat.cpython-310-darwin.so' (no such file), '/Users/vsvh/Library/Python/3.10/lib/python/site-packages/pyreadstat/pyreadstat.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64'))

In [49]:
layers = pd.read_spss("../ukdata/layers.sav")
layers.replace('999', np.nan, inplace=True)
layers.head()

ImportError: Missing optional dependency 'pyreadstat'.  Use pip or conda to install pyreadstat.

In [49]:
layers.columns

Index(['subjectno', 'alter_added_time', 'atype_recode', 'alterno', 'agender',
       'arelation', 'affcontact_days', 'amobile', 'alandline', 'support_1',
       'sympathy_1', 'networklayer_1', 'networklayer_1_recode', 'subjectno2',
       'atype2', 'alterno2', 'TIME_alterenter', 'alterint2', 'agender2',
       'arelation2', 'face_contact_days2', 'amobile2', 'alandline2',
       'aeemotionc2', 'support2', 'sympathy2', 'networklayer_2', 'subjecno_3',
       'related_3', 'alterno_3', 'timeadded_3', 'relationship_3',
       'days_fcontact_3', 'mobile_3', 'landline_3', 'support_3', 'sympathy_3',
       'network_3'],
      dtype='object')

In [50]:
layers.head(2)

,subjectno,alter_added_time,atype_recode,alterno,agender,arelation,affcontact_days,amobile,alandline,support_1,...,related_3,alterno_3,timeadded_3,relationship_3,days_fcontact_3,mobile_3,landline_3,support_3,sympathy_3,network_3
0,1.0,Time 1 (beginnng of study),Related,011,Male,Father,1.0,07968 983808,0114 2685494,Yes,...,Related,011,Time 1 (beginnng of study),Father,5.0,07968 983808,0114 2685494,Yes,Yes,Support clique
1,1.0,Time 1 (beginnng of study),Related,0110,Female,Cousin,87.0,NaN,0038631863362,No,...,Related,0110,Time 1 (beginnng of study),Cousin,83.0,NaN,0038631863362,No,Yes,Sympathy group


In [51]:
dealnum(layers.at[0, 'alandline'])

'1142685494'

In [52]:
layersd = {}
for i in layers.index:
    ego = layers.at[i, 'subjectno'].astype('int64')
    layersd[ego] = layersd.get(ego, {})
    num1 = dealnum(layers.at[i, 'amobile'])
    num1b = dealnum(layers.at[i, 'alandline'])
    if num1 in alterid[ego].keys():
        alter = alterid[ego][num1]
        layersd[ego][alter] = layersd[ego].get(alter, {})
        if not pd.isna(layers.at[i, 'networklayer_1']):
            layersd[ego][alter][1] = dict({'Support clique': 0, 'Sympathy group': 1, 'Active network': 2})[layers.at[i, 'networklayer_1']]
    elif num1b in alterid[ego].keys():
        alter = alterid[ego][num1b]
        layersd[ego][alter] = layersd[ego].get(alter, {})
        if not pd.isna(layers.at[i, 'networklayer_1']):
            layersd[ego][alter][1] = dict({'Support clique': 0, 'Sympathy group': 1, 'Active network': 2})[layers.at[i, 'networklayer_1']]
    num2 = dealnum(layers.at[i, 'amobile2'])
    num2b = dealnum(layers.at[i, 'alandline2'])
    if num2 in alterid[ego].keys():
        alter = alterid[ego][num2]
        layersd[ego][alter] = layersd[ego].get(alter, {})
        if not pd.isna(layers.at[i, 'networklayer_2']):
            layersd[ego][alter][2] = dict({'Support clique': 0, 'Sympathy group': 1, 'Active network': 2})[layers.at[i, 'networklayer_2']]
    elif num2b in alterid[ego].keys():
        alter = alterid[ego][num2b]
        layersd[ego][alter] = layersd[ego].get(alter, {})
        if not pd.isna(layers.at[i, 'networklayer_2']):
            layersd[ego][alter][2] = dict({'Support clique': 0, 'Sympathy group': 1, 'Active network': 2})[layers.at[i, 'networklayer_2']]
    num3 = dealnum(layers.at[i, 'mobile_3'])
    num3b = dealnum(layers.at[i, 'landline_3'])
    if num3 in alterid[ego].keys():
        alter = alterid[ego][num3]
        layersd[ego][alter] = layersd[ego].get(alter, {})
        if not pd.isna(layers.at[i, 'network_3']):
            layersd[ego][alter][3] = dict({'Support clique': 0, 'Sympathy group': 1, 'Active network': 2})[layers.at[i, 'network_3']]
    elif num3b in alterid[ego].keys():
        alter = alterid[ego][num3b]
        layersd[ego][alter] = layersd[ego].get(alter, {})
        if not pd.isna(layers.at[i, 'network_3']):
            layersd[ego][alter][3] = dict({'Support clique': 0, 'Sympathy group': 1, 'Active network': 2})[layers.at[i, 'network_3']]
            
for ego in layersd.keys():
    for alter in layersd[ego].keys():
        for t in range(1, 4):
            if t not in layersd[ego][alter].keys():
                layersd[ego][alter][t] = np.nan

In [53]:
uk['layer1'] = uk['pair3'].map(lambda i: layersd[i[0]][i[1]][1] if i[1] in layersd[i[0]].keys() else np.nan)
uk['layer2'] = uk['pair3'].map(lambda i: layersd[i[0]][i[1]][2] if i[1] in layersd[i[0]].keys() else np.nan)
uk['layer3'] = uk['pair3'].map(lambda i: layersd[i[0]][i[1]][3] if i[1] in layersd[i[0]].keys() else np.nan)

In [50]:
uk.head()

,ego,alter,time,date,isCall,duration,pair,eid,efemale,pair2,...,pair3,afemale,related,sameGen,ec1,ec2,ec3,face1,face2,face3
0,7970136770,7800643277,2007-03-05 18:17:00,2007-03-05,0,0,"(7970136770, 7800643277)",12,0,"(12, 7800643277)",...,"(12, 20)",1.0,0.0,NaN,8.0,2.0,6.0,5.0,16.0,1.0
1,7970136770,7800643277,2007-03-05 18:22:00,2007-03-05,0,0,"(7970136770, 7800643277)",12,0,"(12, 7800643277)",...,"(12, 20)",1.0,0.0,NaN,8.0,2.0,6.0,5.0,16.0,1.0
2,7970136770,7800643277,2007-03-05 18:25:00,2007-03-05,0,0,"(7970136770, 7800643277)",12,0,"(12, 7800643277)",...,"(12, 20)",1.0,0.0,NaN,8.0,2.0,6.0,5.0,16.0,1.0
3,7970136770,7800643277,2007-03-05 18:33:00,2007-03-05,0,0,"(7970136770, 7800643277)",12,0,"(12, 7800643277)",...,"(12, 20)",1.0,0.0,NaN,8.0,2.0,6.0,5.0,16.0,1.0
4,7970136770,7709186078,2007-03-05 18:51:00,2007-03-05,0,0,"(7970136770, 7709186078)",12,0,"(12, 7709186078)",...,"(12, 17)",1.0,0.0,NaN,9.0,9.0,1.0,5.0,23.0,22.5


## Activities 

In [51]:
UKc = pd.read_csv("data/UKc2.txt", sep="\t", dtype={'ego2': str, 'alter2': str})
UKc.head()

,ego,alter,pair,time,date,duration,t,a,efemale,afemale,...,sameGen,ec1,ec2,ec3,face1,face2,face3,pair2,ego2,alter2
0,1,0,"(1, 0)",2007-03-08 11:26:00,2007-03-08 00:00:00,3,3,0,0,1.0,...,0.0,10.0,10.0,8.0,NaN,NaN,NaN,"('7805206865', '7968983808')",7805206865,7968983808
1,1,0,"(1, 0)",2007-03-30 09:48:00,2007-03-30 00:00:00,1,25,22,0,1.0,...,0.0,10.0,10.0,8.0,NaN,NaN,NaN,"('7805206865', '7968983808')",7805206865,7968983808
2,1,0,"(1, 0)",2007-03-31 13:23:00,2007-03-31 00:00:00,2,26,23,0,1.0,...,0.0,10.0,10.0,8.0,NaN,NaN,NaN,"('7805206865', '7968983808')",7805206865,7968983808
3,1,0,"(1, 0)",2007-04-11 09:44:00,2007-04-11 00:00:00,3,37,34,0,1.0,...,0.0,10.0,10.0,8.0,NaN,NaN,NaN,"('7805206865', '7968983808')",7805206865,7968983808
4,1,0,"(1, 0)",2007-04-11 10:49:00,2007-04-11 00:00:00,42,37,34,0,1.0,...,0.0,10.0,10.0,8.0,NaN,NaN,NaN,"('7805206865', '7968983808')",7805206865,7968983808


In [52]:
UKc['pair'].iloc[0]

'(1, 0)'

In [3]:
activities = pd.read_csv("../ukdata/activities.csv", na_values=[9999, 999, " "], sep="\t")

In [5]:
for c in activities.columns:
    print(c)

subjectno
date_time1
location
complete
subjectint
gender
ego_gender_recode
VAR00005
ethnic
dateofb
shefftime
atype
alter_added_time
atype_recode
alterno
alterint
aliveego
agender
alter_gender_recode
aegender
aegender_same
atimeknown
arelation
VAR00006
VAR00014
VAR00011
arelation_genetic
arelation_genetic_recode
dummy_genetic
dummy_affinal
genetic_friends
affinal_friends
alreation_genetic
arelation2_recode
arelation3
affcontact
affcondate
aocontact
aocontype
aocontdate
affcontact_days
affcontact_days_LOG
aocontact_days
aocontact_days_LOG
anycontact_days_days
contact_freq_permonth
contact_freq_peryear
anycontact_days_LOG
amobile
alandline
alocation
alocation_recode
aemotionc_1
aeemotionc_LOG
asubjcl1
asubjcl2
subjclose
subjclose_LOG
asport
aleisure
asocial
awork
aholiday
activitysc
activitysc_LOG
comments
relate_type_copy
VAR00012
type_copy
ec_copy
alterint_copy
support_1
sympathy_1
networklayer_1
subject_copy2
relate_type_copy3
networklayer_1_recode
rmoney_1
rassistance_1
radvice_1
gmon

In [18]:
activities['arelation'].value_counts()

9.0     186
7.0      75
8.0      64
16.0     32
2.0      29
3.0      29
11.0     29
1.0      28
17.0     23
4.0      22
12.0     16
28.0     11
22.0     10
13.0     10
35.0      7
36.0      4
23.0      4
27.0      3
34.0      3
33.0      2
37.0      2
26.0      1
29.0      1
25.0      1
24.0      1
Name: arelation, dtype: int64

In [12]:
activities.head()

,subjectno,date_time1,location,complete,subjectint,gender,ego_gender_recode,VAR00005,ethnic,dateofb,...,live_sheff_3,ec_3,ec_3_recode,EC_CHANGE_3_1,ec_change_3_2,sport_3,leisure_3,social_3,work_3,holiday_3
0,1,NaN,sheffield,1,ah,1.0,1.0,1,0,4/25/1989,...,NaN,8.0,8.0,-2.0,-2.0,0.0,1.0,1.0,0.0,1.0
1,1,03/12/2007,sheffield,1,ah,1.0,1.0,1,0,4/25/1989,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0
2,1,03/12/2007,sheffield,1,ah,1.0,1.0,1,0,4/25/1989,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0
3,1,03/12/2007,sheffield,1,ah,1.0,1.0,1,0,4/25/1989,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0
4,1,03/12/2007,sheffield,1,ah,1.0,1.0,1,0,4/25/1989,...,NaN,8.0,8.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0


In [ ]:
type(activities['amobile'].iloc[0])

In [ ]:
type(UKc['alter2'].iloc[0])

In [ ]:
activities['tmp'] = activities['amobile'].map(lambda i: str(i).replace(" ", "")[-10:])
for i in activities.index:
    ego = activities['subjectno'][i]
    alter = activities['tmp'][i]
    if pd.isna(alter):
        activities['pair'] = pd.NA
    else:
        df = UKc.loc[(UKc['ego'] == ego) & (UKc['alter2'] == alter)]
        if len(df) > 0:
            activities['pair'] = df['pair'].iloc[0]
        else:
            activities['pair'] = pd.NA
# activities['aid'] = activities.index.map(lambda i: alterid[activities['subjectno'][i]][activities['tmp'][i]] if activities['tmp'][i] in alterid[activities['subjectno'][i]].keys() else pd.NA)

In [ ]:
activities['tmp'] = activities['amobile2'].map(lambda i: str(i).replace(" ", "")[-10:])
for i in activities.index:
    if pd.isna(activities['pair'][i]):
        ego = activities['subjectno'][i]
        alter = activities['tmp'][i]
        if pd.isna(alter):
            activities['pair'] = pd.NA
        else:
            df = UKc.loc[(UKc['ego'] == ego) & (UKc['alter2'] == alter)]
            if len(df) > 0:
                activities['pair'] = df['pair'].iloc[0]
            else:
                activities['pair'] = pd.NA

In [ ]:
activities['tmp'] = activities['mobile_3'].map(lambda i: str(i).replace(" ", "")[-10:])
for i in activities.index:
    if pd.isna(activities['pair'][i]):
        ego = activities['subjectno'][i]
        alter = activities['tmp'][i]
        if pd.isna(alter):
            activities['pair'] = pd.NA
        else:
            df = UKc.loc[(UKc['ego'] == ego) & (UKc['alter2'] == alter)]
            if len(df) > 0:
                activities['pair'] = df['pair'].iloc[0]
            else:
                activities['pair'] = pd.NA

In [ ]:
activities['tmp'] = activities['alandline'].map(lambda i: str(i).replace(" ", "")[-10:])
for i in activities.index:
    if pd.isna(activities['pair'][i]):
        ego = activities['subjectno'][i]
        alter = activities['tmp'][i]
        if pd.isna(alter):
            activities['pair'] = pd.NA
        else:
            df = UKc.loc[(UKc['ego'] == ego) & (UKc['alter2'] == alter)]
            if len(df) > 0:
                activities['pair'] = df['pair'].iloc[0]
            else:
                activities['pair'] = pd.NA

In [ ]:
activities['tmp'] = activities['alandline2'].map(lambda i: str(i).replace(" ", "")[-10:])
for i in activities.index:
    if pd.isna(activities['pair'][i]):
        ego = activities['subjectno'][i]
        alter = activities['tmp'][i]
        if pd.isna(alter):
            activities['pair'] = pd.NA
        else:
            df = UKc.loc[(UKc['ego'] == ego) & (UKc['alter2'] == alter)]
            if len(df) > 0:
                activities['pair'] = df['pair'].iloc[0]
            else:
                activities['pair'] = pd.NA

In [ ]:
activities['tmp'] = activities['landline_3'].map(lambda i: str(i).replace(" ", "")[-10:])
for i in activities.index:
    if pd.isna(activities['pair'][i]):
        ego = activities['subjectno'][i]
        alter = activities['tmp'][i]
        if pd.isna(alter):
            activities['pair'] = pd.NA
        else:
            df = UKc.loc[(UKc['ego'] == ego) & (UKc['alter2'] == alter)]
            if len(df) > 0:
                activities['pair'] = df['pair'].iloc[0]
            else:
                activities['pair'] = pd.NA

In [ ]:
sum(~activities['pair'].isna())

In [ ]:
activities['pair'].value_counts()

In [ ]:
# activities['pair'] = list(zip(activities['subjectno'], activities['aid']))
# activities['pair'] = activities['pair'].astype(str)
activities.head()

In [ ]:
activities['pair'][0]

In [ ]:
activities = activities[['pair', 'asport', 'aleisure', 'asocial', 'awork', 'aholiday', 'asport2', 'aleisure2', 'asocial2', 'awork2', 'aholiday2', 'sport_3', 'leisure_3', 'social_3', 'work_3', 'holiday_3']]
activities.columns = ['pair', 'asport1', 'aleisure1', 'asocial1', 'awork1', 'aholiday1', 'asport2', 'aleisure2', 'asocial2', 'awork2', 'aholiday2', 'asport3', 'aleisure3', 'asocial3', 'awork3', 'aholiday3']

In [ ]:
activities

In [ ]:
# with open('data/activities.pickle', 'wb') as acts:
#     pickle.dump(activities, acts, protocol=pickle.HIGHEST_PROTOCOL)
# acts.close()

## Produce data for EC analysis 

In [55]:
tmp = uk.copy(deep=True)
mindate = min(tmp['date'])
tmp['t'] = tmp['date'].map(lambda i: (i - mindate).days)
t0ix = tmp.groupby('pair')['date'].min()
tmp['a'] = tmp.index.map(lambda i: (tmp.at[i, 'date'] - t0ix[tmp.at[i, 'pair']]).days)
tmp.head()

,ego,alter,time,date,isCall,duration,pair,eid,efemale,pair2,...,ec2,ec3,face1,face2,face3,layer1,layer2,layer3,t,a
0,7896556837,7794665331,2007-03-05 15:02:00,2007-03-05,1,56,"(7896556837, 7794665331)",6,0,"(6, 7794665331)",...,8.0,8.0,1.0,1.0,1.0,0.0,0.0,0.0,0,0
1,7896556837,7800650511,2007-03-05 16:29:00,2007-03-05,0,0,"(7896556837, 7800650511)",6,0,"(6, 7800650511)",...,4.0,1.0,5.0,124.0,286.0,2.0,2.0,2.0,0,0
2,7896556837,7821100539,2007-03-05 16:37:00,2007-03-05,1,28,"(7896556837, 7821100539)",6,0,"(6, 7821100539)",...,10.0,9.0,1.0,1.0,1.0,0.0,0.0,0.0,0,0
3,7896556837,7800650511,2007-03-05 16:47:00,2007-03-05,1,2,"(7896556837, 7800650511)",6,0,"(6, 7800650511)",...,4.0,1.0,5.0,124.0,286.0,2.0,2.0,2.0,0,0
4,7896556837,1142551732,2007-03-05 16:48:00,2007-03-05,1,143,"(7896556837, 1142551732)",6,0,"(6, 1142551732)",...,8.0,8.0,1.0,1.0,133.0,0.0,0.0,1.0,0,0


In [56]:
tmp.columns

Index(['ego', 'alter', 'time', 'date', 'isCall', 'duration', 'pair', 'eid',
       'efemale', 'pair2', 'aid', 'pair3', 'afemale', 'related', 'sameGen',
       'ec1', 'ec2', 'ec3', 'face1', 'face2', 'face3', 'layer1', 'layer2',
       'layer3', 't', 'a'],
      dtype='object')

In [58]:
tmp_ell = tmp.groupby('pair')['a'].max()
tmp_ec = tmp.groupby('pair')[['ec1', 'ec2', 'ec3']].first()
tmp_related = tmp.groupby('pair')['related'].first()
tmp_nix = tmp.groupby('pair')['time'].count()
tmp_prop_calls = tmp.groupby('pair')['isCall'].mean()
tmp_eid = tmp.groupby('pair')['eid'].first()
tmp_aid = tmp.groupby('pair')['aid'].first()
tmp_t0ix = tmp.groupby('pair')['t'].min()
tmp_tfix = tmp.groupby('pair')['t'].max()

In [59]:
uk_ec = pd.DataFrame(index=list(tmp['pair'].unique()), columns=['ego', 'eid', 'alter', 'aid', 'ec1', 'ec2', 'ec3', 'related', 'ell', 'nix', 'propcalls', 't0', 'tf'])
for i in list(tmp['pair'].unique()):
    uk_ec.at[i, 'ego'] = i[0]
    uk_ec.at[i, 'eid'] = tmp_eid[i]
    uk_ec.at[i, 'alter'] = i[1]
    uk_ec.at[i, 'aid'] = tmp_aid[i]
    for t in range(1,4):
        uk_ec.at[i, f'ec{t}'] = tmp_ec[f'ec{t}'][i]
    uk_ec.at[i, 'related'] = tmp_related[i]
    uk_ec.at[i, 'ell'] = tmp_ell[i]
    uk_ec.at[i, 'nix'] = tmp_nix[i]
    uk_ec.at[i, 'propcalls'] = tmp_prop_calls[i]
    uk_ec.at[i, 't0'] = tmp_t0ix[i]
    uk_ec.at[i, 'tf'] = tmp_tfix[i]
uk_ec.head()

,ego,eid,alter,aid,ec1,ec2,ec3,related,ell,nix,propcalls,t0,tf
"(7896556837, 7794665331)",7896556837,6,7794665331,6,9.0,8.0,8.0,0.0,293,249,0.522088,0,293
"(7896556837, 7800650511)",7896556837,6,7800650511,63,3.0,4.0,1.0,0.0,280,9,0.333333,0,280
"(7896556837, 7821100539)",7896556837,6,7821100539,0,9.0,10.0,9.0,1.0,527,196,0.566327,0,527
"(7896556837, 1142551732)",7896556837,6,1142551732,7,9.0,8.0,8.0,0.0,504,11,1.0,0,504
"(7896556837, 7707061494)",7896556837,6,7707061494,54,6.0,6.0,2.0,0.0,301,70,0.114286,0,301


In [61]:
# uk_ec.to_csv("data/uk_ec.txt", index=False, header=True, sep="\t")

In [65]:
tmp['t'].max()

545

## Split between calls and texts and save

In [46]:
ukt = uk.loc[uk['isCall'] == 0]
ukc = uk.loc[uk['isCall'] == 1]

In [47]:
uk.head()

,ego,alter,time,date,isCall,duration,pair,eid,efemale,pair2,...,pair3,afemale,related,sameGen,ec1,ec2,ec3,face1,face2,face3
0,7896556837,7794665331,2007-03-05 15:02:00,2007-03-05,1,56,"(7896556837, 7794665331)",6,0,"(6, 7794665331)",...,"(6, 6)",1.0,0.0,NaN,9.0,8.0,8.0,1.0,1.0,1.0
1,7896556837,7800650511,2007-03-05 16:29:00,2007-03-05,0,0,"(7896556837, 7800650511)",6,0,"(6, 7800650511)",...,"(6, 63)",1.0,0.0,NaN,3.0,4.0,1.0,5.0,124.0,286.0
2,7896556837,7821100539,2007-03-05 16:37:00,2007-03-05,1,28,"(7896556837, 7821100539)",6,0,"(6, 7821100539)",...,"(6, 0)",0.0,1.0,0.0,9.0,10.0,9.0,1.0,1.0,1.0
3,7896556837,7800650511,2007-03-05 16:47:00,2007-03-05,1,2,"(7896556837, 7800650511)",6,0,"(6, 7800650511)",...,"(6, 63)",1.0,0.0,NaN,3.0,4.0,1.0,5.0,124.0,286.0
4,7896556837,1142551732,2007-03-05 16:48:00,2007-03-05,1,143,"(7896556837, 1142551732)",6,0,"(6, 1142551732)",...,"(6, 7)",1.0,0.0,NaN,9.0,8.0,8.0,1.0,1.0,133.0


In [48]:
ukt.head()

,ego,alter,time,date,isCall,duration,pair,eid,efemale,pair2,...,pair3,afemale,related,sameGen,ec1,ec2,ec3,face1,face2,face3
1,7896556837,7800650511,2007-03-05 16:29:00,2007-03-05,0,0,"(7896556837, 7800650511)",6,0,"(6, 7800650511)",...,"(6, 63)",1.0,0.0,NaN,3.0,4.0,1.0,5.0,124.0,286.0
5,7896556837,7707061494,2007-03-05 17:37:00,2007-03-05,0,0,"(7896556837, 7707061494)",6,0,"(6, 7707061494)",...,"(6, 54)",1.0,0.0,NaN,6.0,6.0,2.0,1.0,93.0,43.0
6,7896556837,7800755987,2007-03-05 17:37:00,2007-03-05,0,0,"(7896556837, 7800755987)",6,0,"(6, 7800755987)",...,"(6, 9)",1.0,0.0,NaN,7.0,8.0,7.0,1.0,1.0,1.0
7,7896556837,7757845137,2007-03-05 17:38:00,2007-03-05,0,0,"(7896556837, 7757845137)",6,0,"(6, 7757845137)",...,"(6, 14)",1.0,0.0,NaN,8.0,7.0,7.0,1.0,11.0,1.0
8,7896556837,7734730388,2007-03-05 17:39:00,2007-03-05,0,0,"(7896556837, 7734730388)",6,0,"(6, 7734730388)",...,"(6, 57)",0.0,0.0,NaN,4.0,6.0,2.0,5.0,2.0,43.0


In [49]:
counter = 0
for d in [ukt, ukc]:
    df = d.copy(deep=True)
    mindate = min(df['date'])
    df['t'] = df['date'].map(lambda i: (i - mindate).days)
    t0ix = df.groupby('pair3')['date'].min()
    df['a'] = df.index.map(lambda i: (df.at[i, 'date'] - t0ix[df.at[i, 'pair3']]).days)
    df['pair27'] = df['pair']
    df['ego2'] = df['ego']
    df['alter2'] = df['alter']
    df = df[['eid', 'aid', 'pair3', 'time', 'date', 'duration', 't', 'a', 'efemale', 'afemale', 'related', 'sameGen', 'ec1', 'ec2', 'ec3', 'face1', 'face2', 'face3', 'pair27', 'ego2', 'alter2']].rename({'eid': 'ego', 'aid': 'alter', 'pair3': 'pair', 'pair27': 'pair2'}, axis='columns')
    df = df.sort_values(by=['ego', 'alter', 'time']).reset_index(drop=True)
    if counter == 1:
        df.to_csv("data/UKc2.txt", index=False, header=True, sep="\t")
    else:
        df.to_csv("data/UKt2.txt", index=False, header=True, sep="\t")
    counter += 1

In [ ]:
df.head(2)

In [ ]:
len(ukc['pair'].unique())

## Personality Traits

In [ ]:
ptuk = pd.read_excel("../ukdata/personality_t3.xls")
ptuk.head()

In [ ]:
ptuk = ptuk.rename(columns={'subjectno': 'ego'})
ptuk.head()

In [ ]:
ptuk = ptuk.set_index('ego')
ptuk.head()

In [ ]:
ptuk = ptuk.iloc[:, 2:]
print(len(ptuk))
ptuk.head(2)

In [ ]:
personality['uk'] = ptuk

In [ ]:
pwd

In [ ]:
# with open('data/personality.pickle', 'wb') as pers:
#     pickle.dump(personality, pers, protocol=pickle.HIGHEST_PROTOCOL)
# pers.close()

In [ ]:
# with open('data/personality.pickle', 'rb') as handle:
#     aa = pickle.load(handle)

In [ ]:
personality['it']